# Prediction on Persons
Here we are trying to make preditions wheater Persons from out graph database are good or bad. 
To make this we had help from this site: https://www.geeksforgeeks.org/multiclass-classification-using-scikit-learn/

In [31]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
from py2neo import Graph
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC 
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import confusion_matrix 

### Getting the data
We're pulling the persons from the graph

In [2]:
url = "neo4j://localhost:7687"
username = "neo4j"
password = "pleaseletmein"

session = Graph("bolt://localhost:7687", auth=("neo4j", password))

In [3]:
query = ''' MATCH (p:Person) RETURN p.name, p.embedding; '''
result = session.run(query).data()
df = DataFrame(result)

In [4]:
df.columns = ["name", "embedding"]

We are converting it into a csv, inorder to manually add wheather people are good, bad, real or thing.

In [7]:
df.to_csv('HP_Persons.csv')

We're reading the excel sheet where we have altered the data

In [82]:
df = pd.read_excel('HP_Persons.xlsx')

In [83]:
df = df.drop("Column1", axis=1)

In order to make the classification easier and focusing on good and bad, we remove all the Real and thing

In [84]:
df = df.drop(df[df.Status == "Real"].index)

In [85]:
df = df.drop(df[df.Status == "Thing"].index)

We splits the embeddings into each their own cell

In [86]:
df['embedding'] = df.embedding.apply(lambda x: x[1:-1].split(','))

In [87]:
df2 = df.embedding.apply(pd.Series)

In [88]:
df = pd.concat([df, df2], axis=1)

In [89]:
df

,name,Status,embedding,0,1,2,3,4,5,6,...,758,759,760,761,762,763,764,765,766,767
0,Harry,Good,"[0.0554661899805069, 0.08762118965387344, -0...",0.0554661899805069,0.08762118965387344,-0.019243743270635605,0.014201984740793705,-0.009242274798452854,0.03193330019712448,0.003699988592416048,...,0.012820999138057232,-0.010130052454769611,-0.025637757033109665,0.018611906096339226,-0.008647894486784935,0.06780567765235901,0.019771531224250793,0.025388095527887344,-0.03252621740102768,-0.007751178462058306
1,Scabbers,Bad,"[0.0447714626789093, 0.04372702166438103, -0...",0.0447714626789093,0.04372702166438103,-0.006937826052308083,0.00017033978656399995,-0.035803359001874924,0.016571935266256332,-0.012373975478112698,...,0.0062648337334394455,0.0016474747098982334,-0.020052535459399223,0.02419605478644371,-0.013569868169724941,0.041650574654340744,0.01640363596379757,-0.012378434650599957,-0.037346068769693375,-0.0007143528200685978
2,Fred,Good,"[0.04668577387928963, 0.06711672246456146, -...",0.04668577387928963,0.06711672246456146,-0.02943517453968525,0.01713918335735798,-0.0056519899517297745,0.016713475808501244,-0.01353745348751545,...,0.013912532478570938,-0.015418739058077335,-0.043026573956012726,0.02771172672510147,-0.0081111965700984,0.05466648191213608,0.0472407191991806,0.008438504301011562,-0.04130227491259575,-0.010776243172585964
3,Ron,Good,"[0.058472584933042526, 0.07164731621742249, ...",0.058472584933042526,0.07164731621742249,-0.03246584162116051,0.018604660406708717,-0.016124481335282326,0.02757416106760502,-0.019244594499468803,...,0.026459848508238792,0.0009260252118110657,-0.05699407681822777,0.03377788886427879,6.378175021382049e-05,0.05758826434612274,0.038908086717128754,0.006456473376601934,-0.047205470502376556,-0.0043112835846841335
4,Molly Weasley,Good,"[0.056798167526721954, 0.11087218672037125, ...",0.056798167526721954,0.11087218672037125,-0.022338509559631348,0.02829190343618393,0.015429483726620674,0.016161613166332245,-0.011285803280770779,...,0.004324291832745075,0.007377530448138714,0.0016631472390145063,0.01695195399224758,0.004773342050611973,0.08857139199972153,0.007242530584335327,0.00734473904594779,-0.025943312793970108,-0.00823401752859354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,Minerva McGonagall,Good,"[0.055223871022462845, 0.09873385727405548, ...",0.055223871022462845,0.09873385727405548,-0.010511481203138828,0.039570268243551254,0.026631832122802734,-0.005641243886202574,-0.008217244409024715,...,0.003707554657012224,-0.014428874477744102,-0.024516494944691658,0.00019872035773005337,-0.022120941430330276,0.05373264476656914,0.016093198210000992,0.04842901974916458,-0.012857655063271523,-0.00743181724101305
503,Agnes Scribner,Good,"[0.037225402891635895, 0.07300110161304474, ...",0.037225402891635895,0.07300110161304474,-0.0005204884801059961,0.0018217820907011628,0.003944030497223139,0.029815325513482094,-0.0007348418002948165,...,0.0039934986270964146,-0.014426660723984241,-0.0038837501779198647,0.02595781907439232,-0.018637338653206825,-0.008951538242399693,-0.0006648518028669059,-0.0019827643409371376,-0.025268429890275,-0.01809905469417572
504,Victor Ketsueki,Good,"[0.06615018844604492, 0.07716969400644302, -...",0.06615018844604492,0.07716969400644302,-0.01368379220366478,0.0017880717059597373,0.034667108207941055,0.03350356966257095,-0.01840806193649769,...,0.0002878686645999551,0.00021418672986328602,-0.03107842057943344,-0.0003238646895624697,-0.010388427414000034,0.050097834318876266,0.05298904329538345,-0.02661307156085968,-0.027611734345555305,-0.006317094899713993
505,Sebastian Sallow,Bad,"[0.059542685747146606, 0.094327412545681, 7....",0.059542685747146606,0.094327412545681,7.800864841556177e-05,-0.01185739878565073,-0.0008587263291701674,0.003966076299548149,0.006104552652686834,...,0.03337067738175392,-0.011486820876598358,-0.05080188065767288,0.02877655066549778,-0.01903604343533516,-0.012850411236286163,0.0252647064

### Traing and test

We defines X and Y, where X is the embeddings in their own cell, and Y is the label, we're checking

In [90]:
X = df.drop(["name", "Status", "embedding"], axis=1)
Y = df["Status"]

We split the data into train and test set, because we only have 422 persons in it, we have decided to make the train and test set 50/50 instead of the usally 80/20.

In [99]:
x_train, x_test, y_train, y_test= train_test_split(X, Y, test_size=0.5, random_state=101)

### Decision Tree
Here we're using a decision tree classifier, and creating a confusion matrix

In [93]:
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(x_train, y_train) 
dtree_predictions = dtree_model.predict(x_test) 

# creating a confusion matrix 
cm = confusion_matrix(y_test, dtree_predictions) 

In [94]:
cm

array([[  6,  28],
       [ 11, 166]], dtype=int64)

Here can we see that it guesses corret 172 times and only 39 times wrong which isn't that bad

### K-Nearst Neighbors
To see if there was better methods, we tried to make the clasification again with K-nearst neighbors, and made a new confusion matrix

In [118]:
knn = KNeighborsClassifier(n_neighbors = 3).fit(x_train, y_train) 
  
# accuracy on X_test 
accuracy = knn.score(x_test, y_test) 
print (accuracy) 
  
# creating a confusion matrix 
knn_predictions = knn.predict(x_test)  
cm = confusion_matrix(y_test, knn_predictions) 
cm

0.8530805687203792


array([[ 16,  18],
       [ 13, 164]], dtype=int64)

Here we also made an accuracy score, which is 85% and is pretty good. It got 180 right and 31 wrong. 

### SVM classifier 

In [120]:
# training a linear SVM classifier 

svm_model_linear = SVC(kernel = 'linear', C = 1).fit(x_train, y_train) 
svm_predictions = svm_model_linear.predict(x_test) 
  
# model accuracy for X_test   
accuracy = svm_model_linear.score(x_test, y_test) 
print (accuracy) 
   
# creating a confusion matrix 
cm = confusion_matrix(y_test, svm_predictions) 
cm

0.8388625592417062


array([[  0,  34],
       [  0, 177]], dtype=int64)

Here we also made an accuracy score, which is 84%. It got 177 correct and 34 wrong. The funny thing here is that it just but every person as positive, which we guess means good since there are more good persons than bad.

### Naive Bayes classifier

In [122]:
gnb = GaussianNB().fit(x_train, y_train) 
gnb_predictions = gnb.predict(x_test) 
  
# accuracy on X_test 
accuracy = gnb.score(x_test, y_test) 
print (accuracy) 
  
# creating a confusion matrix 
cm = confusion_matrix(y_test, gnb_predictions) 
cm

0.8388625592417062


array([[ 19,  15],
       [ 19, 158]], dtype=int64)